## Pandas 활용한 Market Data Cutting (미완성)
- 아직 내가 원하는데로 Fully Cutting을 못함.
- code가 돌아가는데, 너무 오래 걸림 
    - Why? 
        - 1) pd.read_csv에서 파일 (81.20 MB)을 읽다 보니, 여기에서 많이 잡아먹음.
            - 내 컴퓨터 기준 (around 900ms)
        - 2) Product Code를 찾기 위해서, 다시 A0016 & 001 & BMA 있는 Row를 찾다보니, 또 여기서 시간 잡아먹음.
            - 내 컴퓨터 기준 (around 500ms)
        - 3) Product Code를 기준으로 'A3'와 'G7' Packets만 sorting 하는데 시간이 많이 걸림.
            - 내 컴퓨터 기준 (around 500ms)
- Overall Latency: Approximately 2 seconds
    - 굉장히 느리며, 이유는 1),2),3) 모두 row를 read해야 하기에 오래 걸리는것 같음
    
- 개선방향
    - 사실, pd.read_csv에서 around 900ms 걸린다면, 밑에 만든 Readline을 활용한 MarketData Cutting을 사용하는게 더 나음
    - readline의 경우에는 모든 작업이 끝나는데 400ms 걸림.... 
    - pandas를 활용하면, 훨씬더 빠르다고 들었지만, 그건 아닌것 같음. 파일의 Size, data 형태를 고려해서 선택해야 할 듯 함.|
    
- 갑자기 든 생각 => Value 값들을 String으로 변환 해야 함!!!

In [1]:
%%time
import pandas as pd
import numpy as np
import time

df = pd.read_csv('D:\\Sise_test\\sise2local.ALL_CMDT.0213', 'r', encoding='ISO-8859-1', header=None, names=None, index_col=None)

df[0] = df.iloc[:,0]

KM10_ID = df[(df[0].str.contains('A0016')) & (df[0].str.contains('001')) &
             (df[0].str.contains('BMA'))].iloc[0][0][35:47] # 장운영TS의 국채10년 최근월물

# KM3_id = df[(df[0].str.contains('A0016')) & (df[0].str.contains('001')) & (df[0].str.contains('BM3'))].iloc[0][0][35:47] # 국채 3년
# USD_id = df[(df[0].str.contains('A0016')) & (df[0].str.contains('001')) & (df[0].str.contains('USD'))].iloc[0][0][35:47] # USD선물

# print(KM10_ID)

K10_A3_G7 = df[(df[0].str.contains(KM10_ID)) & ((df[0].str.contains('A3')) | (df[0].str.contains('G7')))] # A3 packet 와 G7 packet grap

K10_A3_G7.to_csv('D:\\Sise_test\\K10_A3_G7.csv', index=False, header=None) # 파일 저장

Wall time: 6.77 s


## Readline 활용한 Market Data Cutting (미완성)
- Readline을 바탕으로 최근에 배운 Pandas를 활용해서 Pandas를 사용했지만, 성능면에서는 Readline이 훨씬 더 빠름.
- 하지만, Pandas는 익숙해지면, readline같은 code를 만드는 것 보다, pandas를 활용해서 하는것이 더 빠른 code를 짜게 할 것 같음.
- Readline에 추가해야 할 것
    - 1) File이 csv로 저장이 되면, sheetname이 file create 할 때, 파일명으로 저장됨.
        - 아마도, csv 파일을 다시 xlsm으로 변환 할 때, sheetname지정하는 기능이 있는걸 보았음. 그걸 활용해서 sheetname을 sheetname1로 변경해야, 지금 만들어져 있는 프로그램이랑 smooth하게 연동 가능 할 듯 함.
    - 2) 파일명: Product Name 및 날짜가 찍히게 수정하기. (Ex.KR4167N30003_0219) 
    - 3) Python folder names in the directory 기능 가능한지 알아보기 
        - Why?
            - 과거 3년치 Data가 있다면, 파일명을 하루하루 바꿔서 하는것은 비효율 적이니, Directory의 filename을 list형태로 변환 후, for 문에 file name을 list로 집어 넣고, 돌리면, 자동으로 돌아가면서 3년치 데이터를 변환 할 수 있을 것 같다.


- 갑자기 든 생각 => Value 값들을 String으로 변환 해야 함!!!

In [153]:
%%time
import numpy as np
import pandas as pd
import time

f = open('D:\\Sise_test\\sise2local.ALL_CMDT.0213', 'r', encoding='ISO-8859-1')
f_1 = open('D:\\Sise_test\\K10_A3_G7_readline.csv', 'a', encoding='ISO-8859-1')
f_1.write('time,cur_price,exe_quantity,short_lvl1_price,short_lvl1_quantity,long_lvl1_price,long_lvl1_quantity,long_or_short,All_short_quantity,All_long_quantity\n')

KTB_productID = ' '

while KTB_productID:
    line = f.readline()
    if (line[17:22] == 'A0016') and (line[425:428] == 'BMA') and (line[471:474] == '001') == True: ## BMA: 10년 국채 Code
        KTB_productID = line[35:47]
        break

while True:
    line = f.readline()

    if not line:
        break

    if (line[17:19] == 'A3') and (line.count(KTB_productID) == 1):
        line = line[0:17] + ',' + line[40:48] + ',' + line[48:54] + ',,,,,' + line[152] + '\n'
        f_1.write(line)

    elif line[17:19] == 'G7' and line.count(KTB_productID):
        line = line[0:17] + ',' + line[40:48] + ',' + line[48:54] + ',' + line[243:251] + ',' + line[-135:-129] + ',' + line[161:169] + ',' + line[169:175] + ',' + line[152] + ',' + line[-151:-144] + ',' + line[153:160] + '\n'
        f_1.write(line)

print(KTB_productID)

f_1.close()
f.close()

data = pd.read_csv('D:\\Sise_test\\K10_A3_G7_readline.csv') # pd.read 대신에 , read기능이나 다른기능이 더 효율적일순 없을까?
data = data.ix[1:]     # row(1) 삭제 하기 위함 - Auction Period 체결량이지만, 지금 활용하고 싶지는 않음
data.to_csv('D:\\Sise_test\\K10_A3_G7_readline.csv', index = False)

KR4167N30003
Wall time: 1.95 s


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [ ]:
%%time
import numpy as np
import pandas as pd
import time

f = open('D:\\Sise_test\\sise2local.ALL_CMDT.0213', 'r', encoding='ISO-8859-1')
f_1 = open('D:\\Sise_test\\K10_A3_G7_readline.csv', 'a', encoding='ISO-8859-1')
f_1.write('time,cur_price,exe_quantity,short_lvl1_price,short_lvl1_quantity,long_lvl1_price,long_lvl1_quantity,long_or_short,All_short_quantity,All_long_quantity\n')

KTB_productID = ' '

while KTB_productID:
    line = f.readline()
    if (line[17:22] == 'A0016') and (line[425:428] == 'BMA') and (line[471:474] == '001') == True: ## BMA: 10년 국채 Code
        KTB_productID = line[35:47]
        break

while True:
    line = f.readline()

    if not line:
        break

    if (line[17:19] == 'A3') and (line.count(KTB_productID) == 1):
        line = line[0:17] + ',' + line[40:48] + ',' + line[48:54] + ',,,,,' + line[152] + '\n'
        f_1.write(line)

    elif line[17:19] == 'G7' and line.count(KTB_productID):
        line = line[0:17] + ',' + line[40:48] + ',' + line[48:54] + ',' + line[243:251] + ',' + line[-135:-129] + ',' + line[161:169] + ',' + line[169:175] + ',' + line[152] + ',' + line[-151:-144] + ',' + line[153:160] + '\n'
        f_1.write(line)

print(KTB_productID)

f_1.close()
f.close()

data = pd.read_csv('D:\\Sise_test\\K10_A3_G7_readline.csv') # pd.read 대신에 , read기능이나 다른기능이 더 효율적일순 없을까?
data = data.ix[1:]     # row(1) 삭제 하기 위함 - Auction Period 체결량이지만, 지금 활용하고 싶지는 않음
data.to_csv('D:\\Sise_test\\K10_A3_G7_readline.csv', index = False)

In [196]:
df = pd.read_csv('D:\\Sise_test\\K10_A3_G7_readline.csv')
df.loc[:, 'long_lvl1_quantity'].fillna(float(N)).astype(int)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'type'

In [151]:
# import numpy as np
# import pandas as pd
# import time

# f = open('D:\\Sise_test\\sise2local.ALL_CMDT.0213', 'r', encoding='ISO-8859-1')
# f_1 = open('D:\\Sise_test\\K10_A3_G7_readline.csv', 'a', encoding='ISO-8859-1')
# f_1.write('time,cur_price,exe_quantity,short_lvl1_price,short_lvl1_quantity,long_lvl1_price,long_lvl1_quantity,long_or_short,All_short_quantity,All_long_quantity\n')

# KTB_productID = ' '

# while KTB_productID:
#     line = f.readline()
#     if (line[17:22] == 'A0016') and (line[425:428] == 'BMA') and (line[471:474] == '001') == True: ## BMA: 10년 국채 Code
#         KTB_productID = line[35:47]
#         break

# while True:
#     line = f.readline()

#     if not line:
#         break

#     if (line[17:19] == 'A3') and (line.count(KTB_productID) == 1):
#         line = line[0:17] + ',' + line[40:48] + ',' + line[48:54] + ',,,,,' + line[152] + '\n'
#         f_1.write(line)

#     elif line[17:19] == 'G7' and line.count(KTB_productID):
#         line = line[0:17] + ',' + line[40:48] + ',' + line[48:54] + ',' + line[243:251] + ',' + line[-135:-129] + ',' + line[161:169] + ',' + line[169:175] + ',' + line[152] + ',' + line[-151:-144] + ',' + line[153:160] + '\n'
#         f_1.write(line)

# print(KTB_productID)

# f_1.close()
# f.close()

# # data = pd.read_csv('D:\\Sise_test\\K10_A3_G7_readline.csv')


# # time	cur_price	exe_quantity	short_lvl1_price	short_lvl1_quantity	long_lvl1_price	long_lvl1_quantity	long_or_short	All_short_quantity	All_long_quantity

# data = pd.read_csv('D:\\Sise_test\\K10_A3_G7_readline.csv', converters={'cur_price':int})
# # pd.to_numeric(s, downcast='float')

# # pd.to_numeric(data.iloc[:,1], downcast='integer', errors='coerce').to_csv('D:\\Sise_test\\test.csv', index=False)
# data.iloc[:,1].to_csv('D:\\Sise_test\\test.csv', index=False, float_format=True)
# # df=pandas.read_csv("C:/Folder/Data.csv",converters={"Price":int})
# # print(data.iloc[:,1].astype(int)) # cur_price int
# # print(data.iloc[:,2].fillna(value=0).astype('int')) # exe_quantity


# # pd.read 대신에 , read기능이나 다른기능이 더 효율적일순 없을까?
# data = data.ix[1:]     # row(1) 삭제 하기 위함 - Auction Period 체결량이지만, 지금 활용하고 싶지는 않음

# # print(data.iloc[:,7].fillna(value=0).astype('int'))
# # numeric = data.iloc[:,7].fillna('0')# pd.to_numeric(s, errors='ignore')
# # numeric.convert_objects(convert_numeric=True).astype(int)
# # print((numeric))
# # df['ColumnName'] = df['ColumnName'].convert_objects(convert_numeric=True) str -> 숫자

KR4167N30003


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
